In [1]:
import pandas as pd



In [2]:
from datasets import load_dataset

dataset = load_dataset("solomonk/reddit_mental_health_posts")
df = dataset['train'].to_pandas()
df.head()

Repo card metadata block was not found. Setting CardData to empty.


,author,body,created_utc,id,num_comments,score,subreddit,title,upvote_ratio,url
0,HotConversation1273,A few months ago I was accepted into this full...,2021-12-22T18:32:56.000Z,rmbjwb,1,1,ADHD,I get extremely anxious if I’m not working 24/7,1.0,https://www.reddit.com/r/ADHD/comments/rmbjwb/...
1,snorefestt,"Hey guys, I was curious if anyone else has the...",2021-12-22T18:24:25.000Z,rmbd1y,3,5,ADHD,"I can't will myself to clean my own house, but...",1.0,https://www.reddit.com/r/ADHD/comments/rmbd1y/...
2,etyf12,\n\ni have 6 exams in the next 2 weeks one of...,2021-12-22T18:22:52.000Z,rmbbvu,1,2,ADHD,i need some help,1.0,https://www.reddit.com/r/ADHD/comments/rmbbvu/...
3,GetHairOrDieTryin,Is there anyone out there that is struggling w...,2021-12-22T18:20:35.000Z,rmba1t,3,2,ADHD,Anyone up for a chat?,1.0,https://www.reddit.com/r/ADHD/comments/rmba1t/...
4,ZeroTransPat,"Whenever I get hungry, I never eat because I d...",2021-12-22T18:18:47.000Z,rmb8lm,2,1,ADHD,Figuring out what to eat sucks,1.0,https://www.reddit.com/r/ADHD/comments/rmb8lm/...


In [3]:
import nltk

In [4]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [5]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


In [6]:
def preprocess_text(text):
    if not isinstance(text, str):
        return ""  # or return text if you want to keep original

    text = text.lower()
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    clean_text = ' '.join(tokens)

    return clean_text


In [7]:
df["clean_text"] = df["body"].apply(preprocess_text)

In [8]:
import nltk
print(nltk.data.path) 

['C:\\Users\\maria/nltk_data', 'C:\\Users\\maria\\anaconda3\\nltk_data', 'C:\\Users\\maria\\anaconda3\\share\\nltk_data', 'C:\\Users\\maria\\anaconda3\\lib\\nltk_data', 'C:\\Users\\maria\\AppData\\Roaming\\nltk_data', 'C:\\nltk_data', 'D:\\nltk_data', 'E:\\nltk_data']


In [9]:
df[["body", "clean_text"]].sample(5)

,body,clean_text
107138,[deleted],deleted
131939,[removed],removed
149951,[deleted],deleted
45260,[deleted],deleted
126519,"At this point, I’m struggling for the will to ...",point im struggling live feel ill worrying tho...


In [10]:
df = df[~df["clean_text"].isin(["deleted", "removed"])]

In [11]:
df = df[df["clean_text"].str.split().str.len() > 3]

In [12]:
df["subreddit"].value_counts()

subreddit
OCD           24859
ADHD          20398
depression    13630
ptsd          13516
aspergers     13223
Name: count, dtype: int64

In [13]:
df[["body", "clean_text"]].sample(5)

,body,clean_text
63330,I usually don't talk but tonight I need to. I ...,usually dont talk tonight need started selfhar...
98557,Please don't comment compulsions!\n\nFor me it...,please dont comment compulsion ambiance noise ...
131906,"Since I was a teenager, whenever I would get r...",since teenager whenever would get really stres...
117436,So basically IV been in therapy since I was 16...,basically iv therapy since started relationshi...
65081,I’ve been struggling with undiagnosed depressi...,ive struggling undiagnosed depression possibly...


In [14]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df["label"] = le.fit_transform(df["subreddit"])

In [15]:
dict(zip(le.classes_, le.transform(le.classes_)))

{'ADHD': 0, 'OCD': 1, 'aspergers': 2, 'depression': 3, 'ptsd': 4}

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf = TfidfVectorizer(max_features=5000)  # You can change max_features as needed

X = tfidf.fit_transform(df["clean_text"])

y = df["label"]

In [17]:
print(X.shape)   # Should be something like (num_samples, 5000)
print(y.shape)

(85626, 5000)
(85626,)


In [18]:
print(tfidf.get_feature_names_out()[:50]) 

['aaron' 'ab' 'abandon' 'abandoned' 'abandonment' 'abilify' 'ability'
 'able' 'abnormal' 'abortion' 'abroad' 'abruptly' 'absence' 'absent'
 'absolute' 'absolutely' 'abstract' 'absurd' 'abt' 'abuse' 'abused'
 'abuser' 'abusing' 'abusive' 'academic' 'academically' 'accent' 'accept'
 'acceptable' 'acceptance' 'accepted' 'accepting' 'access' 'accident'
 'accidentally' 'accommodate' 'accommodation' 'accomplish' 'accomplished'
 'accomplishment' 'according' 'account' 'accountability' 'accountable'
 'accurate' 'accurately' 'accused' 'ache' 'achieve' 'achieved']


In [19]:
from sklearn.model_selection import train_test_split

# 80% training, 20% testing – stratify keeps class proportions
X_train, X_test, y_train, y_test = train_test_split(
    X,                 # your TF-IDF matrix
    y,                 # encoded labels
    test_size=0.20,    # 20 % for testing
    random_state=42,   # reproducible split
    stratify=y         # maintain class balance
)

print("Train shape:", X_train.shape, y_train.shape)
print("Test  shape:", X_test.shape, y_test.shape)

Train shape: (68500, 5000) (68500,)
Test  shape: (17126, 5000) (17126,)


In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Create and train the model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Evaluate performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=le.classes_))

Accuracy: 0.8396006072638094
              precision    recall  f1-score   support

        ADHD       0.87      0.88      0.88      4080
         OCD       0.89      0.89      0.89      4972
   aspergers       0.78      0.77      0.77      2645
  depression       0.75      0.81      0.78      2726
        ptsd       0.86      0.79      0.83      2703

    accuracy                           0.84     17126
   macro avg       0.83      0.83      0.83     17126
weighted avg       0.84      0.84      0.84     17126



In [21]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

nb_model = MultinomialNB(alpha=1.0)        # Laplace smoothing; tweak alpha if you like
nb_model.fit(X_train, y_train)

y_pred_nb = nb_model.predict(X_test)

print("Multinomial NB accuracy:", accuracy_score(y_test, y_pred_nb))
print(classification_report(y_test, y_pred_nb, target_names=le.classes_))

Multinomial NB accuracy: 0.771108256452178
              precision    recall  f1-score   support

        ADHD       0.75      0.83      0.79      4080
         OCD       0.77      0.89      0.83      4972
   aspergers       0.77      0.62      0.69      2645
  depression       0.73      0.71      0.72      2726
        ptsd       0.85      0.68      0.76      2703

    accuracy                           0.77     17126
   macro avg       0.78      0.75      0.76     17126
weighted avg       0.77      0.77      0.77     17126



In [22]:
from sklearn.svm import LinearSVC

svm_model = LinearSVC(C=1.0, class_weight="balanced")   # “balanced” combats any residual skew
svm_model.fit(X_train, y_train)

y_pred_svm = svm_model.predict(X_test)

print("Linear SVM accuracy:", accuracy_score(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm, target_names=le.classes_))

Linear SVM accuracy: 0.8327688894079178
              precision    recall  f1-score   support

        ADHD       0.88      0.86      0.87      4080
         OCD       0.92      0.86      0.89      4972
   aspergers       0.75      0.77      0.76      2645
  depression       0.72      0.82      0.77      2726
        ptsd       0.83      0.81      0.82      2703

    accuracy                           0.83     17126
   macro avg       0.82      0.82      0.82     17126
weighted avg       0.84      0.83      0.83     17126

